## STC Jawwy with SQL

In [4]:
from google.colab import files
uploaded = files.upload()


Saving stc jawwy.xlsb to stc jawwy.xlsb


In [5]:
!pip install pyxlsb
#  قراءة الملف وتحويله لقاعدة بيانات SQLite
import pandas as pd
import sqlite3


In [6]:
# اسم الملف
file_name = "stc jawwy.xlsb"

# قراءة الملف pyxlsb
df = pd.read_excel(file_name, engine='pyxlsb')

# إنشاء قاعدة بيانات مؤقتة في الذاكرة
conn = sqlite3.connect(":memory:")

# تحويل بيانات الإكسل لجدول SQL اسمه jawwy
df.to_sql("jawwy", conn, index=False, if_exists="replace")

# عرض أول 5 صفوف من الجدول للتأكد
pd.read_sql_query("SELECT * FROM jawwy LIMIT 5", conn)

,Column1,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name
0,1,42882,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets
1,3,42876,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana
2,4,42957,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
3,5,42942,19307,The Mermaid Princess,76,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
4,7,42923,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,1,Churchill


In [7]:
# عدد الصفوف في الجدول
pd.read_sql_query("SELECT COUNT(*) as total_rows FROM jawwy;", conn)

,total_rows
0,1048575


In [8]:
pd.read_sql_query("SELECT COUNT(*) as empty_program_name FROM jawwy WHERE program_name IS NULL;", conn)

,empty_program_name
0,0


In [9]:
pd.read_sql_query(" SELECT COUNT(*) as empty_rows FROM jawwy WHERE program_name IS NULL OR duration_seconds IS NULL;", conn)

,empty_rows
0,0


In [28]:
pd.read_sql_query("""
SELECT
  COUNT(*) AS total_views,
  COUNT(DISTINCT user_id_maped) AS total_unique_viewers
FROM jawwy;
""" , conn)

,total_views,total_unique_viewers
0,1048575,11578


In [35]:
pd.read_sql_query("""
SELECT
  ROUND(SUM(duration_seconds) / 3600, 2) AS Total_watch_time_in_houres
FROM jawwy;
""", conn)


,Total_watch_time_in_houres
0,358541.0


In [29]:
pd.read_sql_query("""
SELECT
    program_class,
    SUM(duration_seconds) / 3600 AS total_hours,
    COUNT(DISTINCT user_id_maped) AS total_unique_viewers,
    COUNT(*) AS total_views
FROM jawwy
GROUP BY program_class;

""" , conn)

,program_class,total_hours,total_unique_viewers,total_views
0,MOVIE,103444,11355,488401
1,SERIES/EPISODES,255097,3901,560174


In [32]:
pd.read_sql_query("""
SELECT
  program_genre,
  COUNT(*) AS total_views,
  ROUND( (COUNT(*) * 100.0) / (SELECT COUNT(*) FROM jawwy), 2) AS percentage_of_total_views
FROM jawwy
GROUP BY program_genre
ORDER BY total_views DESC;
""", conn)


,program_genre,total_views,percentage_of_total_views
0,Animation,401730,38.31
1,Action,174155,16.61
2,Drama,155048,14.79
3,Comedy,136577,13.03
4,Horror,75306,7.18
5,Thriller,39601,3.78
6,Family,26668,2.54
7,Biography,16135,1.54
8,Documentary,13291,1.27
9,NOT_DEFINED_IN_UMS,4535,0.43


In [22]:
pd.read_sql_query("""
SELECT
    program_name,
    COUNT(*) AS total_views
FROM jawwy
WHERE program_class = 'MOVIE'
GROUP BY program_name
ORDER BY total_views DESC
LIMIT 5;
""", conn)


,program_name,total_views
0,The Boss Baby,23625
1,Trolls,13609
2,The Adventures of Petey and Friends,9510
3,Moana,7521
4,Baywatch,7332


In [26]:
pd.read_sql_query("""
SELECT
    program_name,
    COUNT(*) AS total_views
FROM jawwy
WHERE TRIM(program_class) = 'SERIES/EPISODES'
GROUP BY program_name
ORDER BY total_views DESC
LIMIT 5;
""", conn)


,program_name,total_views
0,Pingu,17063
1,Oscar's Oasis,7174
2,Nan and Lili Ep.,5038
3,Me and My iblings Ep.,4581
4,Taymour Ep.,3390


In [36]:
pd.read_sql_query("""
SELECT
  CASE
    WHEN hd = 1 THEN 'HD'
    ELSE 'SD'
  END AS quality_type,
  COUNT(*) AS total_views,
  ROUND(SUM(duration_seconds) / 3600, 2) AS total_hours
FROM jawwy
GROUP BY hd
ORDER BY total_views DESC;
""", conn)


,quality_type,total_views,total_hours
0,SD,643539,268364.0
1,HD,405036,90177.0
